### 1. Import library

In [22]:
# import semua library yang digunakan

# catatan ***sesuaikan versi browser dengan chromedriver yang akan didownload***

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
import time
import pandas as pd

### 2. Jalankan browser dan tentukan pencarian

In [ ]:
driver = webdriver.Chrome()

# tentukan URL
linkedin_url = "https://www.linkedin.com/jobs/search/?keywords=manager&location=Indonesia"

# langkah untuk membuka chrome (setting maximize)
driver.maximize_window()
driver.get(linkedin_url) # Open web page

# tentukan banyak data yang akan di scrape
no_of_jobs = 2000
n_scroll = int(no_of_jobs/25)+1
print(n_scroll)
i = 1
driver.execute_script("return document.body.scrollHeight") #scroll ke atas
while i <= n_scroll:
    driver.execute_script("return document.body.scrollHeight")
    time.sleep(2) # tunggu 2 detik, untuk loading 
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #scroll to the bottom of page
    time.sleep(3)
    i = i + 1
    try:
        button = driver.find_element(By.XPATH,"/html/body/div[1]/div/main/section[2]/button")
        button.click()
        time.sleep(1)
        print("load more click")
    except:
        driver.execute_script("return document.body.scrollHeight")
        time.sleep(3)

print ("total jobs :")
jobs = driver.find_element(By.CLASS_NAME,"jobs-search__results-list").find_elements(By.TAG_NAME,'li') # return a list
print(len(jobs))

### 3. Get Main Attributes of each Job Listing
Important notes :

The HTML and CSS element path needs to be checked regularly, because it's possible that it will change in the future
You can also group all the possibly changing web elements in one place
This is not the most efficient code ever. It works but definitely needs improvement, and feel free to do so on your own.

In [24]:
job_id= []
job_title = []
company_name = []
location = []
date = []
job_link = []
for job in jobs:
    job_id0 = job.get_attribute('data-entity-urn')
    job_id.append(job_id0)

    job_title0 = job.find_element(By.CSS_SELECTOR,'h3').get_attribute('innerText')
    job_title.append(job_title0)

    company_name0 = job.find_element(By.CSS_SELECTOR,'h4').get_attribute('innerText')
    company_name.append(company_name0)

    location0 = job.find_element(By.CLASS_NAME,'job-search-card__location').get_attribute('innerText')
    location.append(location0)

    date0 = job.find_element(By.CSS_SELECTOR,'div>div>time').get_attribute('datetime')
    date.append(date0)

    job_link0 = job.find_element(By.CSS_SELECTOR,'a').get_attribute('href')
    job_link.append(job_link0)

In [ ]:
jd = []
seniority = []
emp_type = []
job_func = []
industries = []
for item in range(len(jobs)):
    job_func0=[]
    industries0=[]
    # clicking job to view job details
    try:
        job_click_path = f'/html/body/div[1]/div/main/section[2]/ul/li[{item+1}]/a/div[1]/img'
        job_click = job.find_element(By.XPATH,job_click_path).click()
    except:
        job_click_path = f'/html/body/div[1]/div/main/section[2]/ul/li[{item+1}]/div/a'
    
    print(job_click_path)
    job_click = job.find_element(By.XPATH,job_click_path).click()
    time.sleep(3)
    try:
        jd_path = '/html/body/div[1]/div/section/div[2]/div/section[1]/div/div/section'
        jd0 = job.find_element(By.XPATH,jd_path).get_attribute('innerText')
    except:
        jd_path = '/html/body/div[1]/div/section/div[2]/div/section[2]/div/div/section/div'
    
    jd0 = job.find_element(By.XPATH,jd_path).get_attribute('innerText')
    is_benefit = True
    try:
        assert 'Base pay range' in jd0
    except:
        is_benefit = False
        
    if is_benefit==True :
        jd_path = '/html/body/div[1]/div/section/div[2]/div/section[2]/div/div/section/div'
        jd0 = job.find_element(By.XPATH,jd_path).get_attribute('innerText')
        jd.append(jd0)
        jd_path2 = '/html/body/div[1]/div/section/div[2]/div/section[2]/div'
    else:
        jd.append(jd0)
        jd_path2 = '/html/body/div[1]/div/section/div[2]/div/section[1]/div'

    try:
        seniority_path = jd_path2 + '/ul/li[1]/span'
        seniority_class_name = 'description__job-criteria-text description__job-criteria-text--criteria'
        seniority0 = job.find_element(By.CLASS_NAME,seniority_class_name).get_attribute('innerText')
        seniority.append(seniority0)
    except:
        seniority.append('') #handling if seniority is not available
    
    try:
        emp_type_path = jd_path2 + '/ul/li[2]/span'
        emp_type_class_name = ''
        emp_type0 = job.find_element(By.CLASS_NAME,emp_type_class_name).get_attribute('innerText')
        emp_type.append(emp_type0)
    except:
        emp_type.append('') #handling if employment type is not available
    
    try:
        job_func_path = jd_path2 + '/ul/li[3]/span'
        job_func_class_name = ''
        job_func0 = job.find_element(By.CLASS_NAME,job_func_class_name).get_attribute('innerText')
        job_func.append(job_func0)
    except:
        job_func.append('') #handling if job function is not available
    
    try:
        industries_path = jd_path2 + '/ul/li[4]/span'
        industries_class_name = ''
        emp_type0 = job.find_element(By.CLASS_NAME,industries_class_name).get_attribute('innerText')
        industries.append(industries0)
    except:
        industries.append('')

In [26]:
job_data = pd.DataFrame({'ID': job_id,
                        'Date': date,
                        'Company': company_name,
                        'Title': job_title,
                        'Location': location,
                        'Description' : jd,
                        'Level': seniority,
                        'Type': emp_type,
                        'Function': job_func,
                        'Industry': industries
                        })

In [27]:
job_data.head()

,ID,Date,Company,Title,Location,Description,Level,Type,Function,Industry
0,None,2023-10-13,PT Kino Indonesia Tbk,BUSINESS MANAGER,"Tangerang, Banten, Indonesia",Company Description: PT Kino Indonesia Tbk (Ki...,,,,
1,None,2023-11-22,PT. Indosat Tbk,Staff Service Management,"Jakarta, Indonesia",Role Purpose\n\nManage dedicated account targe...,,,,
2,None,2023-10-18,Kalbe Nutritionals (PT Sanghiang Perkasa),Packaging Development Manager,"Jakarta Timur, Jakarta, Indonesia",Responsibilities\n\n\nCreating packaging for a...,,,,
3,None,2023-11-27,PT. Gudang Garam Tbk,Assistant Brand Manager,"Jakarta, Indonesia",Job Purpose\n\nAssist Brand Manager in impleme...,,,,
4,None,2023-11-29,Kalbe Consumer Health,Brand Manager,"North Jakarta, Jakarta, Indonesia",JOIN US TO BE THE BRIGHTEST STAR\n\nWe Do Inno...,,,,


In [31]:
# pastikan openpyxl library sudah di install.
# tentukan targe folder untuk menyimpan output file
output_file_path = 'lowongan_manager_linkedin_scrape.xlsx'
job_data.to_excel(output_file_path, index = False)

### tahap scrapping selesai sampai disini.
## tutorial  selanjutnya melakukan data cleansing hasil scrapping diatas.